In [1]:
import importlib

if importlib.util.find_spec('librosa') is None:
  ! pip install --user librosa
else:
  print('libraries are already installed')

libraries are already installed


In [2]:
from pathlib import Path
# from scipy.io import wavfile
# import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
# import seaborn as sns
# import matplotlib.pyplot as plt
# from collections import Counter
import numpy as np
# import os
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Reshape
from tensorflow.keras.models import Sequential
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.regularizers import l1, l2
# from transformers import AutoProcessor, TFWav2Vec2Model
# from tensorflow.keras import mixed_precision
from sklearn.metrics import classification_report
import json
import librosa

## Mount Drive

In [3]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [4]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Colab Notebooks/Speech recognition')
  # intermediate_folder = Path('/content/gdrive/MyDrive/Temp/Speech recognition project')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'

Mounted at /content/gdrive


## Read and define model

In [5]:
model = Sequential([Conv2D(32, (2, 2), activation='relu', padding='same', input_shape=(32, 20, 1)),
                    MaxPooling2D((2, 2)),
                    Conv2D(32, (2, 2), activation='relu', padding='same'),
                    MaxPooling2D((2, 2)),
                    Conv2D(32, (2, 2), activation='relu', padding='same'),
                    MaxPooling2D((2, 2)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='relu'),
                    Dropout(.175),
                    Dense(20, activation='softmax')])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 20, 32)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 10, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 10, 32)        4128      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 5, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 5, 32)          4128      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 2, 32)         0

In [6]:
model.load_weights(intermediate_folder / 'best_model_mfcc_cnn.ckp')

## Read data

In [7]:
SAMPLE_RATE = 16000

In [8]:
BATCH_SIZE = 256

In [9]:
# X_train = np.load(intermediate_folder / 'train_main_1_sec_audio.npy').astype(float)
# X_train.shape

In [10]:
# mfcc_train_list = []

# for i in tqdm(range(len(X_train))):
#     mfcc_train_list.append(librosa.feature.mfcc(y=X_train[i], sr=SAMPLE_RATE))

# X_train_mfcc_np = np.stack(mfcc_train_list)
# del mfcc_train_list
# X_train_mfcc_np.shape

In [11]:
X_val = np.load(intermediate_folder / 'val_main_1_sec_audio.npy').astype(float)
X_val.shape

(4619, 16000)

In [12]:
mfcc_val_list = []

for i in tqdm(range(len(X_val))):
    mfcc_val_list.append(librosa.feature.mfcc(y=X_val[i], sr=SAMPLE_RATE))

X_val_mfcc_np = np.stack(mfcc_val_list)
del mfcc_val_list
X_val_mfcc_np.shape

  0%|          | 0/4619 [00:00<?, ?it/s]

(4619, 20, 32)

In [13]:
X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy').astype(float)
X_test.shape

(4689, 16000)

In [14]:
mfcc_test_list = []

for i in tqdm(range(len(X_test))):
    mfcc_test_list.append(librosa.feature.mfcc(y=X_test[i], sr=SAMPLE_RATE))

X_test_mfcc_np = np.stack(mfcc_test_list)
del mfcc_test_list
X_test_mfcc_np.shape

  0%|          | 0/4689 [00:00<?, ?it/s]

(4689, 20, 32)

In [15]:
# y_train_labels = pd.read_csv(intermediate_folder / 'train_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_train_labels.shape

In [16]:
y_val_labels = pd.read_csv(intermediate_folder / 'val_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_val_labels.shape

(4619,)

In [17]:
y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_test_labels.shape

(4689,)

In [18]:
with open(intermediate_folder / 'labels.json', 'r') as file:
    labels_dict = json.load(file)
labels_dict

{'down': 0,
 'eight': 1,
 'five': 2,
 'four': 3,
 'go': 4,
 'left': 5,
 'nine': 6,
 'no': 7,
 'off': 8,
 'on': 9,
 'one': 10,
 'right': 11,
 'seven': 12,
 'six': 13,
 'stop': 14,
 'three': 15,
 'two': 16,
 'up': 17,
 'yes': 18,
 'zero': 19}

In [19]:
# le = LabelEncoder()
# le.fit(y_train_labels)

# y_train = le.transform(y_train_labels)
# y_val = le.transform(y_val_labels)
# y_test = le.transform(y_test_labels)

# del y_test_labels
# del y_train_labels
# del y_val_labels
# y_test.shape

In [20]:
# y_train = np.array([labels_dict[lbl] for lbl in y_train_labels])
y_val = np.array([labels_dict[lbl] for lbl in y_val_labels])
y_test = np.array([labels_dict[lbl] for lbl in y_test_labels])

# del y_train_labels
del y_val_labels
del y_test_labels

In [21]:
# import json
# classes = {cl: i for i, cl in enumerate(le.classes_)}
# print(classes)
# with open(intermediate_folder / 'labels.json', 'w') as fp:
#     json.dump(classes, fp)

## Standardize

In [22]:
MEAN = 33.06522526955586
STD = 142.73227249991973
MEAN, STD

(33.06522526955586, 142.73227249991973)

In [23]:
# X_train_mfcc_np = (X_train - MEAN) / STD
X_val_mfcc_np = (X_val_mfcc_np - MEAN) / STD
X_test_mfcc_np = (X_test_mfcc_np - MEAN) / STD
X_val_mfcc_np.shape, X_test_mfcc_np.shape
# X_train.shape, X_val.shape, X_test.shape

((4619, 20, 32), (4689, 20, 32))

## Predict and metrics

In [24]:
# # y_pred = model.predict(np.expand_dims(X_train_mfcc_np, axis=-1), batch_size=BATCH_SIZE)
# y_pred = model.predict(np.expand_dims(X_train_mfcc_np.transpose(0, 2, 1), axis=-1), batch_size=BATCH_SIZE)
# print((y_pred.argmax(axis=1) == y_train).mean())
# print(classification_report(y_train, y_pred.argmax(axis=1)))

In [25]:
# y_pred = model.predict(np.expand_dims(X_val_mfcc_np, axis=-1), batch_size=BATCH_SIZE)
y_pred = model.predict(np.expand_dims(X_val_mfcc_np.transpose(0, 2, 1), axis=-1), batch_size=BATCH_SIZE)
print((y_pred.argmax(axis=1) == y_val).mean())
print(classification_report(y_val, y_pred.argmax(axis=1)))

19/19 [==============================] - 8s 6ms/step
0.8720502273219312
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       247
           1       0.93      0.92      0.92       226
           2       0.90      0.82      0.86       224
           3       0.92      0.89      0.90       261
           4       0.70      0.70      0.70       231
           5       0.88      0.86      0.87       233
           6       0.89      0.86      0.87       215
           7       0.80      0.79      0.80       235
           8       0.88      0.85      0.86       232
           9       0.85      0.88      0.87       234
          10       0.91      0.89      0.90       209
          11       0.92      0.89      0.91       235
          12       0.89      0.91      0.90       240
          13       0.91      0.90      0.91       241
          14       0.93      0.88      0.91       225
          15       0.93      0.93      0.93       225
         

In [26]:
# y_pred = model.predict(np.expand_dims(X_test_mfcc_np, axis=-1), batch_size=BATCH_SIZE)
y_pred = model.predict(np.expand_dims(X_test_mfcc_np.transpose(0, 2, 1), axis=-1), batch_size=BATCH_SIZE)
print((y_pred.argmax(axis=1) == y_test).mean())
print(classification_report(y_test, y_pred.argmax(axis=1)))

19/19 [==============================] - 0s 6ms/step
0.8639368735338026
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       238
           1       0.91      0.88      0.89       230
           2       0.83      0.88      0.85       241
           3       0.93      0.88      0.90       235
           4       0.68      0.73      0.70       223
           5       0.88      0.88      0.88       249
           6       0.87      0.90      0.89       236
           7       0.78      0.76      0.77       233
           8       0.91      0.83      0.86       243
           9       0.83      0.86      0.85       221
          10       0.86      0.88      0.87       222
          11       0.88      0.85      0.87       233
          12       0.85      0.91      0.88       222
          13       0.93      0.94      0.94       231
          14       0.90      0.88      0.89       234
          15       0.91      0.92      0.91       246
         